In [66]:
%matplotlib inline

import os
import datetime

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

In [67]:
pd. set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 0)

In [68]:
alphavantage_csv_path = "/app/StockPricePredictions/data/alphavantage/time_series_daily_adjusted/AAPL.csv"

In [69]:
df = pd.read_csv(alphavantage_csv_path, low_memory=False)
df.sort_values(by=["date"], ascending=True, inplace=True)

df = df[df["date"]>='2010-01-01']

df["date_time"] = pd.to_datetime(df['date'])

df["day_of_week"] = df["date_time"].dt.dayofweek

df["adjusted_close_shift"] = df["5. adjusted close"].shift(-1)

df["percentage_change"] = (df["adjusted_close_shift"] - df["5. adjusted close"]) / df["5. adjusted close"] * 100.0

df["increase"] = df["percentage_change"].apply(lambda x: 1 if x > 0 else 0)

df.set_index("date", inplace=True)

In [70]:
df.shape[0]

3064

In [71]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3064 entries, 2010-01-04 to 2022-03-04
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   1. open               3064 non-null   float64       
 1   2. high               3064 non-null   float64       
 2   3. low                3064 non-null   float64       
 3   4. close              3064 non-null   float64       
 4   5. adjusted close     3064 non-null   float64       
 5   6. volume             3064 non-null   float64       
 6   7. dividend amount    3064 non-null   float64       
 7   8. split coefficient  3064 non-null   float64       
 8   date_time             3064 non-null   datetime64[ns]
 9   day_of_week           3064 non-null   int64         
 10  adjusted_close_shift  3063 non-null   float64       
 11  percentage_change     3063 non-null   float64       
 12  increase              3064 non-null   int64         
dtypes: datet

In [72]:
df.tail()
# df[df.percentage_change == 0]

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,date_time,day_of_week,adjusted_close_shift,percentage_change,increase
date,,,,,,,,,,,,,
2022-02-28,163.060,165.42,162.43,165.12,165.12,95056629.0,0.0,1.0,2022-02-28,0,163.20,-1.162791,0
2022-03-01,164.695,166.60,161.97,163.20,163.20,83474425.0,0.0,1.0,2022-03-01,1,166.56,2.058824,1
2022-03-02,164.390,167.36,162.95,166.56,166.56,79724750.0,0.0,1.0,2022-03-02,2,166.23,-0.198127,0
2022-03-03,168.470,168.91,165.55,166.23,166.23,76678441.0,0.0,1.0,2022-03-03,3,163.17,-1.840823,0
2022-03-04,164.490,165.55,162.10,163.17,163.17,83819592.0,0.0,1.0,2022-03-04,4,NaN,NaN,0


In [73]:
df.increase.value_counts()

1    1621
0    1443
Name: increase, dtype: int64

In [74]:
# The day of the week with Monday=0, Sunday=6.

# 5 = Saturday
# 6 = Sunday
df.day_of_week.value_counts()

1    629
2    628
3    619
4    614
0    574
Name: day_of_week, dtype: int64

In [75]:
loc = df.index.get_loc('2010-01-05')
loc

print(loc)

df.iloc[loc]["date_time"].strftime("%Y-%m-%d")

1


'2010-01-05'

In [76]:
twint_csv_path = "/app/StockPricePredictions/data/twint/apple_finbert_twint_20100101_20220304.csv"

In [77]:
df_twint = pd.read_csv(twint_csv_path, low_memory=False, lineterminator='\n')

In [78]:
df_twint.head()

,Headline,Stock,Positive,Negative,Neutral
0,Apple 2.0: Tablet: Big iPhone or thin MacBook?,2010-01-01,0.999959,3.629326e-05,0.000005
1,"Apple 2.0: Apple tablet to ship in March, sources say",2010-01-04,0.999997,1.389081e-07,0.000003
2,"Apple Tablet Will Be 10-11 Inches, Ships In March, Says WSJ by @jwyarow",2010-01-04,0.999986,8.988271e-06,0.000005
3,CHART OF THE DAY: Android Taking Wind Out Of iPhone's Sails $GOOG $RIMM,2010-01-04,0.999992,3.523099e-06,0.000004
4,Apple 2.0: How many iPhones did Apple sell?,2010-01-04,0.999959,1.397303e-05,0.000027


In [79]:
df_twint['date'] = pd.to_datetime(df_twint['Stock'])

In [80]:
df_twint["day_of_week"] = df_twint["date"].dt.dayofweek

In [81]:
df_twint.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41056 entries, 0 to 41055
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Headline     41032 non-null  object        
 1   Stock        41056 non-null  object        
 2   Positive     41056 non-null  float64       
 3   Negative     41056 non-null  float64       
 4   Neutral      41056 non-null  float64       
 5   date         41056 non-null  datetime64[ns]
 6   day_of_week  41056 non-null  int64         
dtypes: datetime64[ns](1), float64(3), int64(1), object(2)
memory usage: 2.2+ MB


In [82]:
df_twint.head(1)

,Headline,Stock,Positive,Negative,Neutral,date,day_of_week
0,Apple 2.0: Tablet: Big iPhone or thin MacBook?,2010-01-01,0.999959,0.000036,0.000005,2010-01-01,4


In [83]:
# The day of the week with Monday=0, Sunday=6.

# 5 = Saturday
# 6 = Sunday

df_twint.day_of_week.value_counts()

1    7676
0    7512
2    7467
3    6485
4    6359
6    2817
5    2740
Name: day_of_week, dtype: int64

In [84]:
df_twint.head(1)

,Headline,Stock,Positive,Negative,Neutral,date,day_of_week
0,Apple 2.0: Tablet: Big iPhone or thin MacBook?,2010-01-01,0.999959,0.000036,0.000005,2010-01-01,4


In [85]:
df_twint["in_index"] = df_twint["Stock"].apply(lambda x: True if x in df.index else False)

In [86]:
df_twint[df_twint.in_index==False]["day_of_week"].value_counts()

6    2817
5    2740
0    346 
4    109 
1    54  
3    46  
2    24  
Name: day_of_week, dtype: int64

In [87]:
df.index.get_loc('2010-01-04')


0

In [88]:
from datetime import date, datetime, timedelta

In [89]:
test2 =  '2010-01-09'

dtobj1 = datetime.strptime(test2, "%Y-%m-%d")

print(dtobj1)

days = timedelta(2)

print(days)



dtobj2 = (dtobj1 - days).strftime("%Y-%m-%d")
dtobj2

2010-01-09 00:00:00
2 days, 0:00:00


'2010-01-07'

In [90]:
def update_date(x):

    global ERROR_COUNTER

    if x.in_index == True:
        return x.Stock
    else:
        # try:

            dt_time = datetime.strptime(x.Stock, "%Y-%m-%d")

            # loc = df.index.get_loc(date_part)
            if x.day_of_week == 6: # Sunday to Thursday
                days = timedelta(3)
                idx_lookup = (dt_time - days).strftime("%Y-%m-%d")
                print(idx_lookup)
                try:
                    loc = df.index.get_loc(idx_lookup)
                    return df.iloc[loc]["date_time"].strftime("%Y-%m-%d")
                except Exception as e1:
                    ERROR_COUNTER += 1
                    print(str(e1))
                    return x.Stock
            if x.day_of_week == 5: # Saturday to Thursday
                days = timedelta(2)
                idx_lookup = (dt_time - days).strftime("%Y-%m-%d")
                print(idx_lookup)
                try:
                    loc = df.index.get_loc(idx_lookup)
                    return df.iloc[loc]["date_time"].strftime("%Y-%m-%d")
                except Exception as e2:
                    ERROR_COUNTER += 1
                    print(str(e2))
                    return x.Stock
            if x.day_of_week == 4: # Friday to Thursday
                days = timedelta(1)
                idx_lookup = (dt_time - days).strftime("%Y-%m-%d")
                print(idx_lookup)
                try:
                    loc = df.index.get_loc(idx_lookup)
                    return df.iloc[loc]["date_time"].strftime("%Y-%m-%d")
                except Exception as e3:
                    ERROR_COUNTER += 1
                    print(str(3))
                    return x.Stock
            if x.day_of_week == 0: # Monday to Thursday
                days = timedelta(4)
                idx_lookup = (dt_time - days).strftime("%Y-%m-%d")
                print(idx_lookup)
                try:
                    loc = df.index.get_loc(idx_lookup)
                    return df.iloc[loc]["date_time"].strftime("%Y-%m-%d")
                except Exception as e3:
                    ERROR_COUNTER += 1
                    print(str(3))
                    return x.Stock

            else:
                return x.Stock

        # except Exception as e:
        #     ERROR_COUNTER += 1
        #     print(str(e))
        #     print(x.day_of_week, x.Stock, x.in_index)
        #     return x.Stock
    # loc = df.index.get_loc('2010-01-01')
    # loc



In [91]:
# The day of the week with Monday=0, Sunday=6.

# 5 = Saturday
# 6 = Sunday

ERROR_COUNTER = 0 

df_twint["backfill_date"] = df_twint.apply(update_date, axis=1)


2009-12-31
3
2010-01-21
2010-01-21
2010-01-28
2010-01-28
2010-01-28
2010-02-11
2010-02-11
2010-02-11
2010-02-18
2010-02-18
2010-02-25
2010-03-04
2010-03-04
2010-03-11
2010-03-18
2010-03-25
2010-03-25
2010-03-25
2010-04-01
2010-04-01
2010-04-01
2010-04-01
2010-04-01
2010-04-01
2010-04-01
2010-04-01
2010-04-01
2010-04-08
2010-04-22
2010-04-22
2010-04-29
2010-05-13
2010-05-20
2010-05-27
2010-05-27
2010-05-27
2010-05-27
2010-05-27
2010-05-27
2010-05-27
2010-05-27
2010-06-03
2010-06-10
2010-06-17
2010-06-24
2010-06-24
2010-07-01
2010-07-01
2010-07-08
2010-07-15
2010-07-15
2010-07-22
2010-07-22
2010-07-22
2010-08-05
2010-08-05
2010-08-05
2010-08-05
2010-08-05
2010-08-05
2010-08-05
2010-08-12
2010-08-12
2010-08-12
2010-08-19
2010-08-19
2010-08-19
2010-08-26
2010-09-02
2010-09-02
2010-09-09
2010-09-16
2010-09-16
2010-09-16
2010-09-23
2010-09-23
2010-09-23
2010-09-30
2010-09-30
2010-09-30
2010-09-30
2010-09-30
2010-10-07
2010-10-14
2010-10-14
2010-10-21
2010-10-21
2010-10-21
2010-10-21
2010-10-

In [92]:
ERROR_COUNTER

125

In [93]:
df_twint["in_index"] = df_twint["backfill_date"].apply(lambda x: True if x in df.index else False)

In [94]:
df_twint[df_twint.in_index==False]["day_of_week"].value_counts()

6    75
1    54
5    49
3    46
2    24
4    1 
Name: day_of_week, dtype: int64

In [95]:
df_twint[df_twint.in_index==False].shape

(249, 9)

In [96]:
# df_benzinga["text"] = df_benzinga["title"] + " " + df_benzinga["body"]

In [97]:
# df_benzinga['text'] = np.where(df_benzinga["text"], df_benzinga["title"], df_benzinga["text"])

In [98]:
df_twint = df_twint[["backfill_date", "Positive", "Negative", "Neutral"]]

In [99]:
df_twint.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41056 entries, 0 to 41055
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   backfill_date  41056 non-null  object 
 1   Positive       41056 non-null  float64
 2   Negative       41056 non-null  float64
 3   Neutral        41056 non-null  float64
dtypes: float64(3), object(1)
memory usage: 1.3+ MB


In [100]:
df_twint.head()

,backfill_date,Positive,Negative,Neutral
0,2010-01-01,0.999959,3.629326e-05,0.000005
1,2010-01-04,0.999997,1.389081e-07,0.000003
2,2010-01-04,0.999986,8.988271e-06,0.000005
3,2010-01-04,0.999992,3.523099e-06,0.000004
4,2010-01-04,0.999959,1.397303e-05,0.000027


In [101]:
df_twint.rename(columns={"backfill_date": "date"}, inplace=True)

In [102]:
def clean_tweet(df):
    # replace URLs with a whitespace
    df['text'] = df['text'].str.replace('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ')
    
    return df

In [103]:
# df_twint = clean_tweet(df_twint)

In [104]:
df_twint.set_index("date", inplace=True)

In [105]:
df_twint.head()

,Positive,Negative,Neutral
date,,,
2010-01-01,0.999959,3.629326e-05,0.000005
2010-01-04,0.999997,1.389081e-07,0.000003
2010-01-04,0.999986,8.988271e-06,0.000005
2010-01-04,0.999992,3.523099e-06,0.000004
2010-01-04,0.999959,1.397303e-05,0.000027


In [106]:
df_twint_duplicated_index = df_twint[df_twint.index.duplicated(keep=False)]

In [107]:
df_twint_duplicated_index.shape[0]

40964

In [108]:
df_twint_nonduplicated_index = df_twint[~df_twint.index.duplicated(keep=False)]

In [109]:
df_twint_nonduplicated_index.shape[0]

92

In [110]:
# df_twint.groupby('date')['negative'].mean()

df_twint_groupby = df_twint.groupby("date").agg(
     negative = ("Negative", "mean"),
     nuetral = ("Neutral", "mean"),
     positive = ("Positive", "mean"),
     # compound = ("compound", "mean"),
     # nlikes = ("nlikes", "sum"),
     # nretweets = ("nretweets", "sum"),
     )


In [111]:
df_twint_groupby

,negative,nuetral,positive
date,,,
2010-01-01,0.000036,0.000005,0.999959
2010-01-04,0.000007,0.000010,0.999984
2010-01-05,0.498715,0.001116,0.500169
2010-01-06,0.000052,0.085701,0.914247
2010-01-07,0.457010,0.000224,0.542766
...,...,...,...
2022-02-25,0.283295,0.031205,0.685500
2022-02-28,0.102984,0.000162,0.896855
2022-03-01,0.083351,0.250679,0.665971


In [112]:
df_merge = pd.merge(df, df_twint_groupby, how="left", left_index=True, right_index=True)

In [113]:
df_merge.shape

(3064, 16)

In [114]:
df_merge.day_of_week.value_counts()

1    629
2    628
3    619
4    614
0    574
Name: day_of_week, dtype: int64

In [115]:
# df.index
# df_text.index
df_merge[df_merge.negative.isnull()]
# df_merge.shape

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,date_time,day_of_week,adjusted_close_shift,percentage_change,increase,negative,nuetral,positive
date,,,,,,,,,,,,,,,,
2010-01-14,210.11,210.46,209.0200,209.430,6.403832,15460500.0,0.0,1.0,2010-01-14,3,6.296811,-1.671203,0,NaN,NaN,NaN
2010-01-15,210.93,211.60,205.8700,205.930,6.296811,21216700.0,0.0,1.0,2010-01-15,4,6.575371,4.423833,1,NaN,NaN,NaN
2010-01-29,201.08,202.20,190.2500,192.063,5.872793,44498300.0,0.0,1.0,2010-01-29,4,5.954343,1.388607,1,NaN,NaN,NaN
2010-02-19,201.86,203.20,201.1100,201.670,6.166551,14838200.0,0.0,1.0,2010-02-19,4,6.128207,-0.621808,0,NaN,NaN,NaN
2010-02-22,202.34,202.50,199.1900,200.416,6.128207,13948700.0,0.0,1.0,2010-02-22,0,6.025558,-1.675016,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-28,125.57,125.80,124.5500,124.610,124.083661,71311109.0,0.0,1.0,2021-05-28,4,123.755055,-0.264826,0,NaN,NaN,NaN
2021-09-15,148.56,149.44,146.3700,149.030,148.623917,83281315.0,0.0,1.0,2021-09-15,2,148.384570,-0.161041,0,NaN,NaN,NaN
2021-09-17,148.82,148.82,145.7600,146.060,145.662009,129868824.0,0.0,1.0,2021-09-17,4,142.550511,-2.136108,0,NaN,NaN,NaN


In [116]:
df.shape

(3064, 13)

In [117]:
df_twint_groupby.shape

(3025, 3)

In [118]:
df_merge.fillna(method="ffill", inplace=True)

In [119]:
df_merge[df_merge.negative.isnull()]

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,date_time,day_of_week,adjusted_close_shift,percentage_change,increase,negative,nuetral,positive
date,,,,,,,,,,,,,,,,


In [138]:
df_merge.to_csv("/app/StockPricePredictions/data/alphavantage/time_series_daily_adjusted/AAPL_WITH_TWINT_FINBERT_SA.csv")
# df_merge.head()

In [121]:
# df_merge = df_merge.sample(frac=1).reset_index(drop=True)

In [122]:
X = df_merge[["negative", "nuetral", "positive"]]
y = df_merge["increase"]

In [123]:
y.head()

date
2010-01-04    1
2010-01-05    0
2010-01-06    0
2010-01-07    1
2010-01-08    0
Name: increase, dtype: int64

In [124]:
SPLIT = int(0.9 * len(df_merge))

X_train = X[:SPLIT]
X_test = X[SPLIT:]

y_train = y[:SPLIT]
y_test = y[SPLIT:]



In [125]:
X_train

,negative,nuetral,positive
date,,,
2010-01-04,0.000007,0.000010,0.999984
2010-01-05,0.498715,0.001116,0.500169
2010-01-06,0.000052,0.085701,0.914247
2010-01-07,0.457010,0.000224,0.542766
2010-01-08,0.010157,0.000059,0.989783
...,...,...,...
2020-12-08,0.056716,0.004439,0.938845
2020-12-09,0.391460,0.027822,0.580718
2020-12-10,0.147283,0.051941,0.800776


In [126]:
from sklearn.ensemble import RandomForestClassifier

In [127]:
rfc = RandomForestClassifier(n_estimators=200, criterion="entropy")
rfc.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', n_estimators=200)

In [128]:
preds = rfc.predict(X_test)

In [129]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [130]:
matrix = confusion_matrix(y_test, preds)

In [131]:
matrix

array([[75, 76],
       [73, 83]])

In [132]:
score = accuracy_score(y_test, preds)

In [133]:
score

0.5146579804560261

In [134]:
print(classification_report(y_test, preds))


              precision    recall  f1-score   support

           0       0.51      0.50      0.50       151
           1       0.52      0.53      0.53       156

    accuracy                           0.51       307
   macro avg       0.51      0.51      0.51       307
weighted avg       0.51      0.51      0.51       307



In [135]:
corr_matrix = df_merge[["negative", "nuetral", "positive", "increase"]].corr()
print (corr_matrix)

          negative   nuetral  positive  increase
negative  1.000000 -0.150987 -0.726806  0.019773
nuetral  -0.150987  1.000000 -0.569230 -0.041541
positive -0.726806 -0.569230  1.000000  0.012418
increase  0.019773 -0.041541  0.012418  1.000000


In [136]:
from statsmodels.tsa.stattools import grangercausalitytests

In [137]:
#perform Granger-Causality test
grangercausalitytests(df_merge[["positive", "increase"]], maxlag=[1])


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.0202  , p=0.8869  , df_denom=3060, df_num=1
ssr based chi2 test:   chi2=0.0202  , p=0.8869  , df=1
likelihood ratio test: chi2=0.0202  , p=0.8869  , df=1
parameter F test:         F=0.0202  , p=0.8869  , df_denom=3060, df_num=1


{1: ({'ssr_ftest': (0.02022164779032196, 0.8869291500038903, 3060.0, 1),
   'ssr_chi2test': (0.02024147293521443, 0.8868647432702528, 1),
   'lrtest': (0.020241406053173705, 0.8868649289239112, 1),
   'params_ftest': (0.020221647790084497, 0.8869291500038903, 3060.0, 1.0)},
   array([[0., 1., 0.]])])}